In [ ]:
import sys
import os
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt

import pandas as pd

os.environ["USE_PYGEOS"] = "0"
import geopandas as gpd

sys.path.insert(0, "../src/")

from landprice_merger import (
    zensus_landprice_merger,
    landprice_neighborhood_merger,
    combine_landprice_with_geodata,
)
from csv_merger import combine_csvs
from principal_component_analysis import combine_PCA_datasets


In [ ]:
cities = ["Berlin", "Bremen", "Dresden", "Frankfurt_am_Main", "Köln"]

main_path = os.path.join(os.getcwd(), "..")

path_zensus = os.path.join(main_path, "res", "data", "DLR", "2 Zensus")
path_land = os.path.join(main_path, "res", "data", "DLR", "1 Land Prices")
path_neigh = os.path.join(main_path, "res", "data", "DLR", "3 Neighborhoods")


In [ ]:
df_zensus = combine_csvs(str_path=path_zensus)

In [ ]:
gdf_zensus = combine_PCA_datasets(
    df_zensus=df_zensus, str_city=cities[4], str_path=path_zensus
)

In [ ]:
df_land_prices = combine_csvs(str_path=path_land)

gdf_landprices = combine_landprice_with_geodata(
    df_landprice=df_land_prices, str_city=cities[4], str_path=path_land
)

gdf_landprices_names = landprice_neighborhood_merger(
    gdf_landprice=gdf_landprices, str_city=cities[4], str_path_neigh=path_neigh
)


In [ ]:
gdf_zensus

In [ ]:
result = zensus_landprice_merger(
    gdf_landprices=gdf_landprices_names, gdf_zensus=gdf_zensus
)

In [ ]:
result

In [ ]:
ls_df_cities = []

for i in range(len(cities)):
    
    df_zensus = combine_csvs(str_path=path_zensus)

    gdf_zensus = combine_PCA_datasets(
    df_zensus=df_zensus, str_city=cities[i], str_path=path_zensus
    )

    df_land_prices = combine_csvs(str_path=path_land)

    gdf_landprices = combine_landprice_with_geodata(
        df_landprice=df_land_prices, str_city=cities[i], str_path=path_land
    )

    gdf_landprices_names = landprice_neighborhood_merger(
        gdf_landprice=gdf_landprices, str_city=cities[i], str_path_neigh=path_neigh
    )

    result = zensus_landprice_merger(
    gdf_landprices=gdf_landprices_names, gdf_zensus=gdf_zensus
    )

    result_grouped = result.groupby("Neighborhood_Name").median().reset_index()

    
    land_price_grid = gpd.read_file(path_neigh + "/Neighborhoods_" + cities[i] + ".gpkg")
    result_grouped = result_grouped.merge(land_price_grid, on = ["Neighborhood_Name", "Neighborhood_FID"])
    result_grouped_lm = result_grouped.drop(columns = ["Neighborhood_Name"])

    ls_df_cities.append(result_grouped_lm)


### Berlin

In [ ]:
df_berlin = ls_df_cities[0].drop(columns = ["geometry", "City_Name", "City_Code", "index_right", "Neighborhood_FID", "District_Name"])

In [ ]:
df_berlin = pd.read_csv("D:/GitHub/Geo-paw-sitioning/res/data/Geo-paw-sitioningBerlin.csv").drop(columns = ["geometry", "City_Name", "City_Code", "index_right", "Neighborhood_FID", "District_Name"])

In [ ]:
# calculate the correlation matrix
corr = np.corrcoef(df_berlin)


plt.figure(figsize=(30,20))
# plot the heatmap
sns.heatmap(corr, 
        xticklabels=df_berlin.columns,
        yticklabels=df_berlin.columns)

plt.show

In [ ]:
y = df_berlin["Land_Value"].values.reshape(-1, 1)
X = df_berlin.drop(columns = ["Land_Value"])[["buildings_total_units",
"population_total_units",
"households_total_units",
'Area_Count',
"n_vacant",
"MixedUseArea",
"r_6"]].values #'hhtyp_multiplepers_wout_nuclear',

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)


In [ ]:
df_berlin.drop(columns = ["Land_Value"])[["buildings_total_units",
"population_total_units",
"households_total_units",
'Area_Count',
"n_vacant",
"MixedUseArea",
"r_6"]]

In [ ]:

X2 = sm.add_constant(X_scaled)
est = sm.OLS(y_scaled, X2)
est2 = est.fit()
print(est2.summary())


In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_scaled, y_scaled)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

In [ ]:
df_reg_berlin = {
    "Name": ["buildings_total_units",
            "population_total_units",
            "households_total_units",
            'Area_Count',
            "n_vacant",
            "MixedUseArea",
            "r_6"],
    "Coeff": regr.coef_[0].tolist()
}
pd.DataFrame(df_reg_berlin).to_csv("Berlin_coeff.csv")


In [ ]:
scores = cross_val_score(regr, X_scaled, y_scaled, cv = 5)
scores

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

### Bremen

In [ ]:
df_bremen = ls_df_cities[1].drop(columns = ["geometry", "City_Name", "City_Code", "index_right", "Neighborhood_FID", "District_Name", 
"Neighborhood_Code", "District_Code"])

In [ ]:
#df_bremen
df_bremen = pd.read_csv("D:/GitHub/Geo-paw-sitioning/res/data/Geo-paw-sitioningBremen.csv").drop(columns = ["geometry", "City_Name", "City_Code", "index_right", "Neighborhood_FID", "District_Name", 
"Neighborhood_Code", "District_Code"])

In [ ]:
corr = np.corrcoef(df_bremen)


plt.figure(figsize=(30,20))
# plot the heatmap
sns.heatmap(corr, 
        xticklabels=df_bremen.columns,
        yticklabels=df_bremen.columns)

plt.show

In [ ]:
df_bremen.columns

In [ ]:
y = df_bremen["Land_Value"].values.reshape(-1, 1)
X = df_bremen.drop(columns = ["Land_Value"])[["buildings_total_units",
"population_total_units",
"households_total_units",
'Area_Count',
"AgriculturalArea",
"famgr_6_more",
"al_80_over"]].values #'hhtyp_multiplepers_wout_nuclear',

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)


In [ ]:
X2 = sm.add_constant(X_scaled)
est = sm.OLS(y_scaled, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_scaled, y_scaled)

In [ ]:
df_reg_bremen = {
    "Name": ["buildings_total_units",
            "population_total_units",
            "households_total_units",
            'Area_Count',
            "AgriculturalArea",
            "famgr_6_more",
            "al_80_over"],
    "Coeff": regr.coef_[0].tolist()
}
pd.DataFrame(df_reg_bremen).to_csv("Bremen_coeff.csv")

In [ ]:
scores = cross_val_score(regr, X_scaled, y_scaled, cv = 5)
scores

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

### Dresden

In [ ]:
df_dresden = ls_df_cities[2].drop(columns = ["geometry", "City_Name", "City_Code", "index_right", "Neighborhood_FID", "Neighborhood_Code"])

In [ ]:
df_dresden = pd.read_csv("D:/GitHub/Geo-paw-sitioning/res/data/Geo-paw-sitioningDresden.csv").drop(columns = ["geometry", "City_Name", "City_Code", "index_right", "Neighborhood_FID", "Neighborhood_Code"])

In [ ]:
corr = np.corrcoef(df_dresden)


plt.figure(figsize=(30,20))
# plot the heatmap
sns.heatmap(corr, 
        xticklabels=df_dresden.columns,
        yticklabels=df_dresden.columns)

plt.show

In [ ]:
df_dresden.columns

In [ ]:
y = df_dresden["Land_Value"].values.reshape(-1, 1)
X = df_dresden.drop(columns = ["Land_Value"])[["buildings_total_units",
"population_total_units",
"households_total_units",
'Area_Count',
"sz_mult_german_foreign"]].values #'hhtyp_multiplepers_wout_nuclear',

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)

In [ ]:
df_dresden

In [ ]:
X2 = sm.add_constant(X_scaled)
est = sm.OLS(y_scaled, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_scaled, y_scaled)

In [ ]:
df_reg_dresden = {
    "Name": ["buildings_total_units",
    "population_total_units",
    "households_total_units",
    'Area_Count',
    "sz_mult_german_foreign"],
    "Coeff": regr.coef_[0].tolist()
}
pd.DataFrame(df_reg_dresden).to_csv("Dresden_coeff.csv")

In [ ]:
scores = cross_val_score(regr, X_scaled, y_scaled, cv = 5)
scores

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

### Frankfurt am Main

In [ ]:
df_frankfurt = ls_df_cities[3].drop(columns = ["geometry", "City_Name", "City_Code", "Neighborhood_FID"])

In [ ]:
df_frankfurt = pd.read_csv("D:/GitHub/Geo-paw-sitioning/res/data/Geo-paw-sitioningFrankfurt_am_Main.csv").drop(columns = ["geometry", "City_Name", "City_Code", "Neighborhood_FID"])

In [ ]:
corr = np.corrcoef(df_frankfurt)


plt.figure(figsize=(30,20))
# plot the heatmap
sns.heatmap(corr, 
        xticklabels=df_frankfurt.columns,
        yticklabels=df_frankfurt.columns)

plt.show

In [ ]:
df_frankfurt.columns

In [ ]:
y = df_frankfurt["Land_Value"].values.reshape(-1, 1)
X = df_frankfurt.drop(columns = ["Land_Value"])[[
"e_community_of_apartment_owners"]].values #'hhtyp_multiplepers_wout_nuclear',

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_scaled, y_scaled)

In [ ]:
df_reg_frankfurt = {
    "Name": [
"e_community_of_apartment_owners"],
    "Coeff": regr.coef_[0].tolist()
}
pd.DataFrame(df_reg_frankfurt).to_csv("Frankfurt_coeff.csv")

In [ ]:
X2 = sm.add_constant(X_scaled)
est = sm.OLS(y_scaled, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
scores = cross_val_score(regr, X_scaled, y_scaled, cv = 5)
scores

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

### Koeln

In [ ]:
df_koeln = ls_df_cities[4].drop(columns = ["geometry", "City_Name", "City_Code", "Neighborhood_FID", "District_Name"])

In [ ]:
df_koeln = pd.read_csv("D:/GitHub/Geo-paw-sitioning/res/data/Geo-paw-sitioningKöln.csv").drop(columns = ["geometry", "City_Name", "City_Code", "Neighborhood_FID", "District_Name"])

In [ ]:
corr = np.corrcoef(df_koeln)


plt.figure(figsize=(30,20))
# plot the heatmap
sns.heatmap(corr, 
        xticklabels=df_koeln.columns,
        yticklabels=df_koeln.columns)

plt.show

In [ ]:
df_koeln.columns

In [ ]:
y = df_koeln["Land_Value"].values.reshape(-1, 1)
X = df_koeln.drop(columns = ["Land_Value"])[["buildings_total_units",
"al_60_69",
"we_public_companies",
'famk_mothers_children_more_18',
"gg_detached_twofamily"]].values #'hhtyp_multiplepers_wout_nuclear',

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)

In [ ]:
X2 = sm.add_constant(X_scaled)
est = sm.OLS(y_scaled, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_scaled, y_scaled)

df_reg_koeln = {
    "Name": ["buildings_total_units",
"al_60_69",
"we_public_companies",
'famk_mothers_children_more_18',
"gg_detached_twofamily"],
    "Coeff": regr.coef_[0].tolist()
}
pd.DataFrame(df_reg_koeln).to_csv("koeln_coeff.csv")

In [ ]:
scores = cross_val_score(regr, X_scaled, y_scaled, cv = 5)
scores

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))